# Proyecto Final
## Data Science I: Fundamentos para la Ciencia de Datos
### Fecha: 16-07-20204
### Nombre archivo: `TMDB_tv_dataset_v3.csv`
#### source: https://www.kaggle.com/datasets/asaniczka/full-tmdb-tv-shows-dataset-2023-150k-shows?resource=download


## Objetivos
- Elegir un método de feature selection para reducir la dimensionalidad del dataset.
- Elegir un algoritmo de regresión o clasificación para entrenar con los datos elegidos.
- Validar modelo a traves de la metricas basicas. 
- Generar conclusiones con base en los resultados obtenidos.

## Metadata 
El conjunto de datos proporciona información crucial para analizar las tendencias y predecir el éxito de películas y shows de televisión utilizando información de The Movie Database (TMDB). La base de datos esta compuesta por 29 variables y mas de 150000 registros, los cuales incluyen atributos interesantes de analisis como el genero, popularidad y fecha de emision, permitiendo la segmentación y análisis detallado. Esta cantidad de datos permite una comprensión profunda y detallada de las preferencias y comportamientos de la audiencia.

Los análisis derivados de este dataset presentan una oportunidad valiosa para la industria del entretenimiento, permitiendo la identificación de factores que impulsan la popularidad, la evaluación de la efectividad de estrategias de lanzamiento y marketing, y la formulación de recomendaciones personalizadas para los usuarios, por mencionar algunos.

Este conjunto de datos es una herramienta invaluable para investigadores y profesionales de la industria del entretenimiento, contribuyendo a la mejora de la toma de decisiones estratégicas, la personalización de la experiencia del usuario y el incremento de la satisfacción y retención de la audiencia. 

##  Variables
id: id unico del show
 <br> name: nombre del show
 <br> number_of_seasons: numero de temporadas
 <br> number_of_episodes: numero de episodios 
 <br> original_language: idioma original
 <br> vote_count: cantidad de votos, 0 indica sin informacion
 <br> vote_average: nota promedio de los votos, 0 indica sin informacion
 <br> overview: resumen
 <br> adult: boolean que indica si el show es categoria adulto
 <br> backdrop_path: url para la imagen del show
 <br> first_air_date: primera fecha de emision
 <br> last_air_date: ultima fecha de emision 
 <br> homepage: url oficial del show
 <br> in_production: boolean que indica si esta en producion
 <br> original_name: nombre original 
 <br> popularity: popularidad, 0 indica sin informacion
 <br> poster_path: url para la imagen del poster
 <br> type: tipo de show
 <br> status: status 
 <br> tagline: linea asociada al show
 <br> genres: listado de genero
 <br> created_by: creado por
 <br> languages:listado idiomas disponibles
 <br> networks: listado de cadenas que han emitido el programa 
 <br> origin_country: listado de pais/es de origen
 <br> spoken_languages: listado de idiomas hablados 
 <br> production_companies: listado de compañias productoras
 <br> production_countries:listado de los paises de produccion
 <br> episode_run_time: duracion de los episodios, 0 indica no informacion

# Carga de datos

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter, AutoDateLocator,AutoDateFormatter,YearLocator
import seaborn as sns
import statsmodels.formula.api as sm
from sklearn.preprocessing import StandardScaler


In [3]:
tmdb=pd.read_csv("../data/TMDB_tv_dataset_v3.csv")

Normalizacion de variables numericas

In [45]:
numeric_features =["num_Temporadas","num_Episodios","Votos_Totales","Voto_promedio","Popularidad","Duracion_episodio","Mes","estacion","scripted","anho","num_genero","num_idioma"]
X = tmdb[numeric_features]
y = tmdb['Popularidad']
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
df_normalized = pd.DataFrame(X_normalized, columns=numeric_features)

# Análisis exploratorio de datos (EDA)
A continuacion se entregara un resumen de los puntos mas importantes obtenido en el EDA, si se desea ver en mayor detalle puede consultar el notebook ProyectoDS_ParteI_jofre.ipynb

### Renomabramiento de vaiables

In [4]:
nombres_columnas= {'id':"id",
'name':"Nombre_Serie",
'number_of_seasons':"num_Temporadas",
'number_of_episodes':"num_Episodios",
'original_language':"Idioma_original",
'vote_count':"Votos_Totales",
'vote_average':"Voto_promedio",
'overview':"Resumen",
'adult':"Categoria_Adulto",
'backdrop_path':"url_Imagen",
'first_air_date':"Fecha_Inicio",
'last_air_date':"Fecha_Fin",
'homepage':"Pagina",
'in_production':"En_Produccion",
'original_name':"Nombre_Original",
'popularity':"Popularidad",
'poster_path':"url_Poster",
'type':"Tipo_Show",
'status':"Status",
'tagline':"Frase",
'genres':"Generos",
'created_by':"Creador",
'languages':"Idiomas",
'networks':"Canales",
'origin_country':"Pais_Origen",
'spoken_languages':"Idiomas_hablados",
'production_companies':"Companias_Produccion",
'production_countries':"Paises_Produccion",
'episode_run_time':"Duracion_episodio"}

tmdb.rename(columns=nombres_columnas,inplace=True)

### Tratamiento de valores nulos
- Para la columna de generos, se rellena la columna con el valor "Sin información"
- Se eliminan las columnas que tienen soobre un 35% de valores nulos, que no entregan informacion relevante, como url de la imagen del poster. 
- Se eliminan las filas que no poseen valores de las fechas de inicio o final.
- Aún con esto se posee una base con sobre 90 mil filas de información.  

In [5]:
tmdb=tmdb.dropna(subset=['Nombre_Serie'])
tmdb['Generos']=tmdb['Generos'].fillna('sin informacion')
tmdb=tmdb.drop(columns=['Nombre_Serie','Resumen','url_Imagen','url_Poster','Frase','Pagina','Creador'])
tmdb=tmdb.dropna(subset=['Fecha_Inicio','Fecha_Fin','Idiomas' ])
tmdb.shape

(98775, 22)

### Columna calculadas y transformaciòn de categorias
Para obtener un mejor detalle se extrae informacion temporal de las fechas, como el mes de ingreso y la estacion del año.

In [6]:
tmdb['Fecha_Inicio']= pd.to_datetime(tmdb['Fecha_Inicio'])
tmdb['Mes']= tmdb['Fecha_Inicio'].dt.month
tmdb['estacion']=tmdb['Mes']%4+1
tmdb['estacion_n']= tmdb['estacion'].map({1:'primavera',2:'verano',3:'otoño',4:'invierno'})
tmdb['scripted']=tmdb['Generos'].apply(lambda x: 1 if x=='Scripted' else 0)
tmdb['anho']= tmdb['Fecha_Inicio'].dt.year

Ademas dado que el tipo de programa esta desbalanceado a los del tipo script se crea una nueva variable que indica si el programa es del tipo scripted o no.

In [7]:
tmdb['scripted']=tmdb['Generos'].apply(lambda x: 1 if x=='Scripted' else 0)

Por ultimo se agrega una columna indicando cuantos generos utilizo el programa.

In [23]:
def contar_generos (genero):
    if genero=='sin informacion':
        return 0
    else:
        return genero.count(',')+1
tmdb['num_genero']= tmdb['Generos'].apply(contar_generos)
tmdb['num_idioma']=(tmdb['Idiomas'].apply(contar_generos))

In [22]:
(tmdb['Idiomas'].apply(contar_generos)).value_counts()

Idiomas
1     93795
2      4084
3       653
4       142
5        52
6        27
7         8
8         7
10        3
9         2
11        1
12        1
Name: count, dtype: int64

## 

## Reduccion de dimensionalidad

Para poder utilizar tanto la reduccion de dimensionalidad, como la regresion es necesario normalizar los datos para los valores numericos.

In [46]:
numeric_features =["num_Temporadas","num_Episodios","Votos_Totales","Voto_promedio","Popularidad","Duracion_episodio","Mes","estacion","scripted","anho","num_genero","num_idioma"]
X = tmdb[numeric_features]
y = tmdb['Popularidad']
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)
df_normalized = pd.DataFrame(X_normalized, columns=numeric_features)
tmdb[numeric_features]=df_normalized

In [48]:
tmdb_featured= tmdb.drop(columns=['id','Status','Fecha_Fin','Fecha_Inicio','Idioma_original','Nombre_Original','Tipo_Show','Generos','Idiomas','Canales','Pais_Origen','Idiomas_hablados','Companias_Produccion','Paises_Produccion','estacion_n'])
tmdb_featured.head()

,num_Temporadas,num_Episodios,Votos_Totales,Voto_promedio,Categoria_Adulto,En_Produccion,Popularidad,Duracion_episodio,Mes,estacion,scripted,anho,num_genero,num_idioma
0,1.694237,0.218726,87.729459,1.329557,False,False,19.896556,-0.572644,-0.648852,-1.386466,0.0,0.076434,1.899249,-0.198145
1,0.284243,0.031113,71.573326,1.279634,False,False,1.615199,0.752729,-0.366935,-0.450068,0.0,0.504048,0.801406,-0.198145
2,0.566242,-0.009928,64.843277,1.378670,False,True,3.269339,-0.572644,0.196898,1.422727,0.0,0.432779,1.899249,-0.198145
3,2.540234,0.828471,61.914200,1.242934,False,False,8.897508,0.222580,1.042648,0.486330,0.0,0.005165,1.899249,-0.198145
4,1.130240,0.335985,55.638179,1.341431,False,False,7.544719,0.279381,-1.494602,-0.450068,0.0,0.432779,0.801406,-0.198145


In [49]:
# Preprocesado y modelado
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
import statsmodels.api as sm

In [50]:
# Entrenamiento modelo PCA con escalado de los datos
pca_pipe = make_pipeline(StandardScaler(), PCA())
pca_pipe.fit(df_normalized)
modelo_pca = pca_pipe.named_steps['pca']
modelo_pca.explained_variance_ratio_

array([0.16421684, 0.12174627, 0.09835893, 0.09398646, 0.08943393,
       0.08851217, 0.08476692, 0.08132934, 0.06866506, 0.06151631,
       0.04746775, 0.        ])

In [24]:
model1 = 'Popularidad~num_idioma+Idioma_original+Categoria_Adulto+En_Produccion+estacion_n+Mes+scripted+Status+Votos_Totales+Voto_promedio+num_Temporadas+num_Episodios+num_genero+anho'
lm1   = sm.ols(formula = model1, data = tmdb).fit()
print(lm1.summary())

                            OLS Regression Results                            
Dep. Variable:            Popularidad   R-squared:                       0.172
Model:                            OLS   Adj. R-squared:                  0.171
Method:                 Least Squares   F-statistic:                     178.3
Date:                Wed, 17 Jul 2024   Prob (F-statistic):               0.00
Time:                        07:23:36   Log-Likelihood:            -5.2488e+05
No. Observations:               98775   AIC:                         1.050e+06
Df Residuals:                   98659   BIC:                         1.051e+06
Df Model:                         115                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           